In [1]:
# Importing Libraries & Functions
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import re
import warnings
warnings.filterwarnings('ignore')

First, we load in our GAMES and TEAMS datasets and perform some simple cleaning to make them more usable.

In [2]:
# Loading + Cleaning Data 
games = pd.read_csv('FP1_DATA/games.csv')
timestamp = pd.to_datetime(games['GAME_DATE_EST'])
games.insert(0, 'TIMESTAMP', timestamp)
season_list = np.arange(2007,2019)
games = games[games['SEASON'].isin(season_list)].sort_values('TIMESTAMP')

teams = pd.read_csv('FP1_DATA/teams.csv')
teams = teams[['TEAM_ID','NICKNAME']]
team_dict = teams.set_index('TEAM_ID').T.to_dict('list')
games['HOME_TEAM_NAME'] = games['HOME_TEAM_ID'].map(team_dict)
games['AWAY_TEAM_NAME'] = games['VISITOR_TEAM_ID'].map(team_dict)


drop_cols = ['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 
             'VISITOR_TEAM_ID', 'TEAM_ID_home', 'TEAM_ID_away']
games = games.drop(drop_cols, axis = 1).reset_index(drop = True)
front_cols = ['TIMESTAMP', 'GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME']
temp_games = games[front_cols]
games = pd.concat([temp_games, games.drop(front_cols, axis = 1)], axis = 1)
games['HOME_TEAM_NAME'] = games['HOME_TEAM_NAME'].apply(lambda x: x[0])
games['AWAY_TEAM_NAME'] = games['AWAY_TEAM_NAME'].apply(lambda x: x[0])

### Player-Position Statistics

Because we want to consider player/position contributions to the overall statistics, we use the DETAILS dataframe to convert players to positions, and then calculate the statistics in each game, grouped by each position, Guards, Forwards, and Center. We also then calculate Player Efficiency Rating (PER) using these statistics.

In [33]:
# Defining the Position Stat Function
def pos_stat(stat_list):
    
    
    ### Takes in a list of statistics to consider from the DETAILS data frame and calculates
    ### the stats by position in each home for the home and away team.
    ### NOTE: THESE ARE NOT CUMULATIVE STATS!!!!!
    

    # Removing non-starters, getting rid of unnecessary columns
    details = pd.read_csv('FP1_DATA/games_details.csv')
    season_dict = games[['GAME_ID', 'SEASON']].set_index('GAME_ID').iloc[:,0].T.to_dict()
    details['SEASON'] = details['GAME_ID'].map(season_dict)
    details = details[details.notna()]
    details = details[~details['START_POSITION'].isna()]
    details['TEAM_NAME'] = details['TEAM_ID'].map(team_dict).apply(lambda x: x[0])
    details['MINS'] = details['MIN'].str.split(':').apply(lambda x: float(x[0]) + (float(x[1])/60))
    details = details.drop(['TEAM_ABBREVIATION', 'TEAM_CITY', 'COMMENT', 'MIN'], axis = 1)
    temp_cols = ['GAME_ID','START_POSITION', 'TEAM_NAME']
    final_cols = np.append(temp_cols, stat_list).flatten()
    details = details[final_cols]
    
    output_df = pd.DataFrame()
    for stat in stat_list: 
        
        
        # Groupby the GAME, POSITION and TEAM and add up all stats by each position 
        df = details.copy()
        df = df.groupby(['GAME_ID', 'START_POSITION', 'TEAM_NAME']).sum().reset_index()
        df = pd.merge(df, games[['GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME']], on = 'GAME_ID')

        # Need to Re-Order the DF s.t. home and away are known 
        home_condition = (df['TEAM_NAME'] == df['HOME_TEAM_NAME'])
        away_condition = (df['TEAM_NAME'] != df['HOME_TEAM_NAME'])

        # Adding HOME Columns Manually 
        df['C_' + str(stat) + '_home'] = df[ home_condition & (df['START_POSITION'] == 'C')][stat]
        df['F_' + str(stat) + '_home'] = df[ home_condition & (df['START_POSITION'] == 'F')][stat]
        df['G_' + str(stat) + '_home'] = df[ home_condition & (df['START_POSITION'] == 'G')][stat]

        # Adding AWAY Columns Manually 
        df['C_' + str(stat) + '_away'] = df[ away_condition & (df['START_POSITION'] == 'C')][stat]
        df['F_' + str(stat) + '_away'] = df[ away_condition & (df['START_POSITION'] == 'F')][stat]
        df['G_' + str(stat) + '_away'] = df[ away_condition & (df['START_POSITION'] == 'G')][stat]

        # Grouping Again to Get Rid of NaN 
        df = df.groupby('GAME_ID', as_index = False).sum()
        if stat_list.index(stat) > 0: 
            df = df.drop('GAME_ID', axis = 1)
        # Append to final data frame 
        output_df = pd.concat([output_df, df], axis = 1)

    output_df = output_df.drop(stat_list, axis = 1)
    return output_df

In [34]:
# Adding Columns for Position Stats using pos_stat Function
stat_list = ['FGM', 'FGA','FG_PCT', 'FG3M', 'FG3A', 
             'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
             'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 
             'PF', 'PTS', 'PLUS_MINUS']
pos_df = pos_stat(stat_list)

In [35]:
# Calculing Player Efficiency Rating
# Formula: (PTS + REB + AST + STL + BLK − Missed FG − Missed FT - TO) / GP
p1 = pos_df['C_PTS_home'] + pos_df['C_REB_home'] + pos_df['C_AST_home'] + pos_df['C_STL_home']
missed_fg = (pos_df['C_FGA_home'] - pos_df['C_FGM_home'])
missed_ft = pos_df['C_FTA_home'] - pos_df['C_FTM_home']
p2 = (missed_fg + missed_ft + pos_df['C_TO_home'])
per = (p1 - p2)/82

for pos in ['C', 'F', 'G']: 
    p1 = pos_df[pos + '_PTS_home'] + pos_df[pos + '_REB_home'] + pos_df[pos + '_AST_home'] + pos_df[pos + '_STL_home']
    missed_fg = (pos_df[pos + '_FGA_home'] - pos_df[pos + '_FGM_home'])
    missed_ft = pos_df[pos + '_FTA_home'] - pos_df[pos + '_FTM_home']
    p2 = (missed_fg + missed_ft + pos_df[pos + '_TO_home'])
    per = (p1-p2)/82
    pos_df[pos + '_PER_home'] = per
    
for pos in ['C', 'F', 'G']: 
    p1 = pos_df[pos + '_PTS_away'] + pos_df[pos + '_REB_away'] + pos_df[pos + '_AST_away'] + pos_df[pos + '_STL_away']
    missed_fg = (pos_df[pos + '_FGA_away'] - pos_df[pos + '_FGM_away'])
    missed_ft = pos_df[pos + '_FTA_away'] - pos_df[pos + '_FTM_away']
    p2 = (missed_fg + missed_ft + pos_df[pos + '_TO_away'])
    per = (p1-p2)/82
    pos_df[pos + '_PER_away'] = per

### Calculating the Cumulative Statistics


The bulk of our cumulative sum and average calculations are shown below. We first define functions to calculate the cumulative average statistics for the (1) Season, (2) Last 5 Games, and (3) Last 10 Games. Because a team's win or loss count is calculated as a sum, we also create functions to calculate those sums for the Season, Last 5 and Last 10 Games based off of the 'HOME_TEAM_WINS' column.

In [36]:
# Defining the Cumulative Stat Functions
def cum_avg(arr): 
    temp_list = [np.mean(arr[:i]) for i in np.arange(len(arr)+1)][1:]
    return np.append(np.nan, temp_list)[:-1]

def cum_5_avg(arr): 
    means = []
    for i in np.arange(len(arr)): 
        if i < 5: 
            means = np.append(means, np.nan)
        else: 
            means = np.append(means, np.mean(arr[i-5: i+1]))
    return means

def cum_10_avg(arr): 
    means = []
    for i in np.arange(len(arr)): 
        if i < 10: 
            means = np.append(means, np.nan)
        else: 
            means = np.append(means, np.mean(arr[i-10: i+1]))
    return means

def cum_wins(arr): 
    temp_list = [np.sum(arr[:i]) for i in np.arange(len(arr)+1)][1:]
    return np.append(np.nan, temp_list)[:-1]

def cum_5_wins(arr): 
    wins = []
    for i in np.arange(len(arr)): 
        if i < 5: 
            wins = np.append(wins, np.nan)
        else: 
            wins = np.append(wins, np.sum(arr[i-5: i+1]))
    return wins

def cum_10_wins(arr): 
    wins = []
    for i in np.arange(len(arr)): 
        if i < 10: 
            wins = np.append(wins, np.nan)
        else: 
            wins = np.append(wins, np.sum(arr[i-10: i+1]))
    return wins

def cum_losses(arr): 
    temp_list = [np.count_nonzero(arr[:i]==0) for i in np.arange(len(arr)+1)][1:]
    return np.append(np.nan, temp_list)[:-1]

def cum_5_losses(arr): 
    losses = []
    for i in np.arange(len(arr)): 
        if i < 5: 
            losses = np.append(losses, np.nan)
        else: 
            losses = np.append(losses, np.count_nonzero(arr[i-5: i+1]==0))
    return losses

def cum_10_losses(arr): 
    losses = []
    for i in np.arange(len(arr)): 
        if i < 10: 
            losses = np.append(losses, np.nan)
        else: 
            losses = np.append(losses, np.count_nonzero(arr[i-10: i+1]==0))
    return losses

The next two cells do the same thing but for the HOME and AWAY statistics, respectively. For each HOME and AWAY stat, we do the following: 

(1) Calculate the Cumulative Season Average

(2) Calculate the Cumulative Last 5 Game Average

(3) Calculate the Cumulative Last 10 Game Average

(4) Calculate the Cumulative Season Average for this specific combination of HOME and AWAY TEAM (Head-to-Head)

(5) Calculate the Cumulative Last 5 Game Average for this specific combination of HOME and AWAY TEAM (Head-to-Head)

(6) Calculate the Cumulative Last 10 Game Average for this specific combination of HOME and AWAY TEAM (Head-to-Head)


We had to go about this in a rather unorthodox way using two grouped dataframes because using a nested for loop increased the run time by over 50x. We then combine these two dataframes together, using 'GAME_ID' to join. This turns our initial dataframe of 132 columns to 792 features.

In [37]:
# Cumulative Statistics: HOME
df = games.copy()
df = df.merge(pos_df, on = 'GAME_ID')
df['TEAM_LIST'] = df['HOME_TEAM_NAME'] + ', ' + df['AWAY_TEAM_NAME']
df_home = df.drop('AWAY_TEAM_NAME', axis = 1)
df_home = df.sort_values('TIMESTAMP', ascending = True)
column_list = df_home.columns[4:]
home_stat_list = [stat for stat in column_list if 'home' in stat]

t = time.time()
for stat in home_stat_list: 
    
    keep_cols = np.append(['TIMESTAMP', 'TEAM_LIST','GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME'], stat)
    df_home_temp = df_home[keep_cols]
    
    # Two Grouped DFs
    df_1 = df_home_temp.groupby(['HOME_TEAM_NAME', 'GAME_ID']).mean()
    df_2 = df_home_temp.groupby(['HOME_TEAM_NAME']).agg(list)

    # Using Functions above to calulate the cumulative values as lists
    cum_lists = df_2.apply(lambda x: cum_avg(x[stat]), axis = 1)
    cum_5_lists = df_2.apply(lambda x: cum_5_avg(x[stat]), axis = 1)
    cum_10_lists = df_2.apply(lambda x: cum_10_avg(x[stat]), axis = 1)

    # Converting these nested lists to single lists
    cum_vals = cum_lists.to_frame().explode(0)[0].to_numpy()
    cum_5_vals = cum_5_lists.to_frame().explode(0)[0].to_numpy()
    cum_10_vals = cum_10_lists.to_frame().explode(0)[0].to_numpy()

    df_1 = df_1.reset_index()
    df_1['cum_' + str(stat)] = cum_vals
    df_1['cum_5_' + str(stat)] = cum_5_vals
    df_1['cum_10_' + str(stat)] = cum_10_vals
    
    # Converting to Dictionaries and Adding to Large DF
    cum_dict = df_1.set_index('GAME_ID').iloc[:, -3].T.to_dict()
    cum_dict_5 = df_1.set_index('GAME_ID').iloc[:, -2].T.to_dict()
    cum_dict_10 = df_1.set_index('GAME_ID').iloc[:, -1].T.to_dict()

    df_home['cum_' + str(stat)] = df['GAME_ID'].map(cum_dict)
    df_home['cum_5_' + str(stat)] = df['GAME_ID'].map(cum_dict_5)
    df_home['cum_10_' + str(stat)] = df['GAME_ID'].map(cum_dict_10)
    
    
    # HEAD TO HEAD STATS ==> SAME THING BASICALLY 
    
    # Two Grouped DFs
    df_1_H2H = df_home.groupby(['TEAM_LIST', 'GAME_ID']).mean()
    df_2_H2H = df_home.groupby('TEAM_LIST').agg(list)
    
    # Using Functions above to calulate the cumulative values as lists
    cum_lists_H2H = df_2_H2H.apply(lambda x: cum_avg(x[stat]), axis = 1)
    cum_5_lists_H2H = df_2_H2H.apply(lambda x: cum_5_avg(x[stat]), axis = 1)
    cum_10_lists_H2H = df_2_H2H.apply(lambda x: cum_10_avg(x[stat]), axis = 1)

    # Converting these nested lists to single lists
    cum_vals_H2H = cum_lists_H2H.to_frame().explode(0)[0].to_numpy()
    cum_5_vals_H2H = cum_5_lists_H2H.to_frame().explode(0)[0].to_numpy()
    cum_10_vals_H2H = cum_5_lists_H2H.to_frame().explode(0)[0].to_numpy()

    #df_1_H2H = df_1_H2H.reset_index()
    df_1_H2H['cum_' + str(stat) + '_H2H'] = cum_vals_H2H
    df_1_H2H['cum_5_' + str(stat) + '_H2H'] = cum_5_vals_H2H
    df_1_H2H['cum_10_' + str(stat) + '_H2H'] = cum_10_vals_H2H
    
   
    # Converting to Dictionaries and Adding to Large DF
    cum_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -3].T.to_dict()
    cum_5_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -2].T.to_dict()
    cum_10_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -1].T.to_dict()
    
    df_home['cum_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_dict_H2H)
    df_home['cum_5_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_5_dict_H2H)
    df_home['cum_10_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_10_dict_H2H)
  
    
    

    
    print('# ' + str(home_stat_list.index(stat)+1) + ' of ' + str(len(home_stat_list)) + ' COMPLETE: ', str(stat), '-- TIME: ', time.time()-t)


# 1 of 66 COMPLETE:  PTS_home -- TIME:  4.732546091079712
# 2 of 66 COMPLETE:  FG_PCT_home -- TIME:  9.695076942443848
# 3 of 66 COMPLETE:  FT_PCT_home -- TIME:  14.761112928390503
# 4 of 66 COMPLETE:  FG3_PCT_home -- TIME:  19.93681502342224
# 5 of 66 COMPLETE:  AST_home -- TIME:  25.567740201950073
# 6 of 66 COMPLETE:  REB_home -- TIME:  30.941532135009766
# 7 of 66 COMPLETE:  C_FGM_home -- TIME:  36.46212697029114
# 8 of 66 COMPLETE:  F_FGM_home -- TIME:  42.01122498512268
# 9 of 66 COMPLETE:  G_FGM_home -- TIME:  47.68910884857178
# 10 of 66 COMPLETE:  C_FGA_home -- TIME:  53.532379150390625
# 11 of 66 COMPLETE:  F_FGA_home -- TIME:  59.75461792945862
# 12 of 66 COMPLETE:  G_FGA_home -- TIME:  65.97572493553162
# 13 of 66 COMPLETE:  C_FG_PCT_home -- TIME:  72.31214499473572
# 14 of 66 COMPLETE:  F_FG_PCT_home -- TIME:  78.80450510978699
# 15 of 66 COMPLETE:  G_FG_PCT_home -- TIME:  85.44951796531677
# 16 of 66 COMPLETE:  C_FG3M_home -- TIME:  92.15903091430664
# 17 of 66 COMPLETE: 

In [38]:
# Cumulative Statistics: AWAY
df_away = df.drop('HOME_TEAM_NAME', axis = 1)
df_away = df.sort_values('TIMESTAMP', ascending = True)
column_list = df_away.columns[4:]
away_stat_list = [stat for stat in column_list if 'away' in stat]

t = time.time()
for stat in away_stat_list: 
    
    keep_cols = np.append(['TIMESTAMP', 'TEAM_LIST','GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME'], stat)
    df_away_temp = df_away[keep_cols]
    
    # Two Grouped DFs
    df_1 = df_away_temp.groupby(['AWAY_TEAM_NAME', 'GAME_ID']).mean()
    df_2 = df_away_temp.groupby(['AWAY_TEAM_NAME']).agg(list)

    # Using Functions above to calulate the cumulative values as lists
    cum_lists = df_2.apply(lambda x: cum_avg(x[stat]), axis = 1)
    cum_5_lists = df_2.apply(lambda x: cum_5_avg(x[stat]), axis = 1)
    cum_10_lists = df_2.apply(lambda x: cum_10_avg(x[stat]), axis = 1)

    # Converting these nested lists to single lists
    cum_vals = cum_lists.to_frame().explode(0)[0].to_numpy()
    cum_5_vals = cum_5_lists.to_frame().explode(0)[0].to_numpy()
    cum_10_vals = cum_10_lists.to_frame().explode(0)[0].to_numpy()

    df_1 = df_1.reset_index()
    df_1['cum_' + str(stat)] = cum_vals
    df_1['cum_5_' + str(stat)] = cum_5_vals
    df_1['cum_10_' + str(stat)] = cum_10_vals
    
    # Converting to Dictionaries and Adding to Large DF
    cum_dict = df_1.set_index('GAME_ID').iloc[:, -3].T.to_dict()
    cum_dict_5 = df_1.set_index('GAME_ID').iloc[:, -2].T.to_dict()
    cum_dict_10 = df_1.set_index('GAME_ID').iloc[:, -1].T.to_dict()

    df_away['cum_' + str(stat)] = df['GAME_ID'].map(cum_dict)
    df_away['cum_5_' + str(stat)] = df['GAME_ID'].map(cum_dict_5)
    df_away['cum_10_' + str(stat)] = df['GAME_ID'].map(cum_dict_10)

    
    
    
    # HEAD TO HEAD STATS ==> SAME THING BASICALLY 
    
    
    
    # Two Grouped DFs
    df_1_H2H = df_away.groupby(['TEAM_LIST', 'GAME_ID']).mean()
    df_2_H2H = df_away.groupby('TEAM_LIST').agg(list)
    
    # Using Functions above to calulate the cumulative values as lists
    cum_lists_H2H = df_2_H2H.apply(lambda x: cum_avg(x[stat]), axis = 1)
    cum_5_lists_H2H = df_2_H2H.apply(lambda x: cum_5_avg(x[stat]), axis = 1)
    cum_10_lists_H2H = df_2_H2H.apply(lambda x: cum_10_avg(x[stat]), axis = 1)

    # Converting these nested lists to single lists
    cum_vals_H2H = cum_lists_H2H.to_frame().explode(0)[0].to_numpy()
    cum_5_vals_H2H = cum_5_lists_H2H.to_frame().explode(0)[0].to_numpy()
    cum_10_vals_H2H = cum_5_lists_H2H.to_frame().explode(0)[0].to_numpy()

    #df_1_H2H = df_1_H2H.reset_index()
    df_1_H2H['cum_' + str(stat) + '_H2H'] = cum_vals_H2H
    df_1_H2H['cum_5_' + str(stat) + '_H2H'] = cum_5_vals_H2H
    df_1_H2H['cum_10_' + str(stat) + '_H2H'] = cum_10_vals_H2H
    
   
    # Converting to Dictionaries and Adding to Large DF
    cum_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -3].T.to_dict()
    cum_5_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -2].T.to_dict()
    cum_10_dict_H2H = df_1_H2H.droplevel(0).iloc[:, -1].T.to_dict()
    
    df_away['cum_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_dict_H2H)
    df_away['cum_5_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_5_dict_H2H)
    df_away['cum_10_' + str(stat) + '_H2H'] = df['GAME_ID'].map(cum_10_dict_H2H)
    
    
    print('# ' + str(away_stat_list.index(stat)+1) + ' of ' + str(len(away_stat_list)) + ' COMPLETE: ', str(stat), '-- TIME: ', time.time()-t)


# 1 of 66 COMPLETE:  PTS_away -- TIME:  4.816655158996582
# 2 of 66 COMPLETE:  FG_PCT_away -- TIME:  9.50899600982666
# 3 of 66 COMPLETE:  FT_PCT_away -- TIME:  14.484194040298462
# 4 of 66 COMPLETE:  FG3_PCT_away -- TIME:  19.607362985610962
# 5 of 66 COMPLETE:  AST_away -- TIME:  24.847387075424194
# 6 of 66 COMPLETE:  REB_away -- TIME:  30.26409387588501
# 7 of 66 COMPLETE:  C_FGM_away -- TIME:  35.743788957595825
# 8 of 66 COMPLETE:  F_FGM_away -- TIME:  41.35866117477417
# 9 of 66 COMPLETE:  G_FGM_away -- TIME:  47.139140129089355
# 10 of 66 COMPLETE:  C_FGA_away -- TIME:  52.970470905303955
# 11 of 66 COMPLETE:  F_FGA_away -- TIME:  58.956490993499756
# 12 of 66 COMPLETE:  G_FGA_away -- TIME:  65.04403519630432
# 13 of 66 COMPLETE:  C_FG_PCT_away -- TIME:  71.5125629901886
# 14 of 66 COMPLETE:  F_FG_PCT_away -- TIME:  78.20596814155579
# 15 of 66 COMPLETE:  G_FG_PCT_away -- TIME:  84.69293427467346
# 16 of 66 COMPLETE:  C_FG3M_away -- TIME:  91.42034006118774
# 17 of 66 COMPLETE:

In [39]:
# Combining the Home and Away DataFrames 
df_home_2 = df_home.copy()
df_away_2 = df_away.copy()
df_home_2 = df_home_2.drop(np.append(home_stat_list, away_stat_list,), axis = 1)
df_away_2 = df_away_2.drop(np.append(home_stat_list, away_stat_list), axis = 1)

df_clean = pd.merge(df_home_2, df_away_2, on = 'GAME_ID')
xcol = [col for col in df_clean.columns if '_x' in col]
ycol = [col for col in df_clean.columns if '_y' in col]
df_clean = df_clean.drop(ycol, axis = 1)
df_clean.columns = df_clean.columns.str.replace(r'_x$', '')

### Win & Loss Counters


We then do the same thing but for the Win and Loss counter. 

In [40]:
# Win-Loss Counter: HOME
df_clean2 = df_clean.copy()
season_dict = games[['GAME_ID', 'SEASON']].set_index('GAME_ID').iloc[:,0].T.to_dict()
df_clean2['SEASON'] = df_clean2['GAME_ID'].map(season_dict)

keep_cols = ['TIMESTAMP', 'SEASON', 'HOME_TEAM_WINS','GAME_ID', 'HOME_TEAM_NAME', 'AWAY_TEAM_NAME']
df_clean2_temp = df_clean2[keep_cols]
df_clean2_temp['AWAY_TEAM_WINS'] = df_clean2_temp['HOME_TEAM_WINS'].map({0:1, 1:0})

# Two Grouped DFs
df_g1 = df_clean2_temp.groupby(['HOME_TEAM_NAME', 'SEASON','GAME_ID']).mean()
df_g2 = df_clean2_temp.groupby(['HOME_TEAM_NAME']).agg(list)

# Using Functions above to calulate the cumulative values as lists

    # HOME WINS!
cum_wins_home = df_g2['HOME_TEAM_WINS'].apply(cum_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_5_wins_home = df_g2['HOME_TEAM_WINS'].apply(cum_5_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_10_wins_home = df_g2['HOME_TEAM_WINS'].apply(cum_10_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()

    # HOME WINS!
cum_losses_home = df_g2['HOME_TEAM_WINS'].apply(cum_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_5_losses_home = df_g2['HOME_TEAM_WINS'].apply(cum_5_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_10_losses_home = df_g2['HOME_TEAM_WINS'].apply(cum_10_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()


# #df_clean = df_clean.reset_index()
df_clean2['cum_WINS_home'] = cum_wins_home
df_clean2['cum_LOSSES_home'] = cum_losses_home
df_clean2['cum_5_WINS_home'] = cum_5_wins_home
df_clean2['cum_10_WINS_home'] = cum_10_wins_home
df_clean2['cum_5_LOSSES_home'] = cum_5_losses_home
df_clean2['cum_10_LOSSES_home'] = cum_10_losses_home


In [41]:
# Win-Loss Counter: AWAY


df_clean2_temp['AWAY_TEAM_WINS'] = df_clean2_temp['HOME_TEAM_WINS'].map({0:1, 1:0})

# Two Grouped DFs
df_g1 = df_clean2_temp.groupby(['AWAY_TEAM_NAME', 'SEASON','GAME_ID']).mean()
df_g2 = df_clean2_temp.groupby(['AWAY_TEAM_NAME']).agg(list)

# Using Functions above to calulate the cumulative values as lists

    # AWAY WINS!
cum_wins_away = df_g2['AWAY_TEAM_WINS'].apply(cum_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_5_wins_away = df_g2['AWAY_TEAM_WINS'].apply(cum_5_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_10_wins_away = df_g2['AWAY_TEAM_WINS'].apply(cum_10_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()

    # AWAY LOSSES
cum_losses_away = df_g2['AWAY_TEAM_WINS'].apply(cum_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_5_losses_away = df_g2['AWAY_TEAM_WINS'].apply(cum_5_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_10_losses_away = df_g2['AWAY_TEAM_WINS'].apply(cum_10_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()


# #df_clean = df_clean.reset_index()
df_clean2['cum_WINS_away'] = cum_wins_away
df_clean2['cum_LOSSES_away'] = cum_losses_away
df_clean2['cum_5_WINS_away'] = cum_5_wins_away
df_clean2['cum_10_WINS_away'] = cum_10_wins_away
df_clean2['cum_5_LOSSES_away'] = cum_5_losses_away
df_clean2['cum_10_LOSSES_away'] = cum_10_losses_away


In [42]:
# Win-Loss Counter: HOME HEAD-TO-HEAD 

#HOME HEAD TO HEAD Win/Loss Counter

df_clean2_temp['AWAY_TEAM_WINS'] = df_clean2_temp['HOME_TEAM_WINS'].map({0:1, 1:0})
df_clean2_temp['TEAM_LIST'] = df_clean2_temp['HOME_TEAM_NAME'] + ', ' + df_clean2_temp['AWAY_TEAM_NAME']


# Two Grouped DFs
df_g1 = df_clean2_temp.groupby(['TEAM_LIST', 'SEASON','GAME_ID']).mean()
df_g2 = df_clean2_temp.groupby(['TEAM_LIST']).agg(list)

# Using Functions above to calulate the cumulative values as lists

    # HOME WINS!
cum_wins_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_5_wins_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_5_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_10_wins_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_10_wins).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()

    # HOME Losses!
cum_losses_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_5_losses_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_5_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()
cum_10_losses_home_H2H = df_g2['HOME_TEAM_WINS'].apply(cum_10_losses).to_frame().explode('HOME_TEAM_WINS')['HOME_TEAM_WINS'].to_numpy()


# #df_clean = df_clean.reset_index()
df_clean2['cum_WINS_home_H2H'] = cum_wins_home_H2H
df_clean2['cum_LOSSES_home_H2H'] = cum_losses_home_H2H
df_clean2['cum_5_WINS_home_H2H'] = cum_5_wins_home_H2H
df_clean2['cum_10_WINS_home_H2H'] = cum_10_wins_home_H2H
df_clean2['cum_5_LOSSES_home_H2H'] = cum_5_losses_home_H2H
df_clean2['cum_10_LOSSES_home_H2H'] = cum_10_losses_home_H2H

In [43]:
# Win-Loss Counter: AWAY HEAD-TO-HEAD

# Two Grouped DFs
df_g1 = df_clean2_temp.groupby(['TEAM_LIST', 'SEASON','GAME_ID']).mean()
df_g2 = df_clean2_temp.groupby(['TEAM_LIST']).agg(list)

# Using Functions above to calulate the cumulative values as lists

    # HOME WINS!
cum_wins_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_5_wins_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_5_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_10_wins_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_10_wins).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()

    # HOME Losses!
cum_losses_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_5_losses_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_5_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()
cum_10_losses_away_H2H = df_g2['AWAY_TEAM_WINS'].apply(cum_10_losses).to_frame().explode('AWAY_TEAM_WINS')['AWAY_TEAM_WINS'].to_numpy()


# #df_clean = df_clean.reset_index()
df_clean2['cum_WINS_away_H2H'] = cum_wins_away_H2H
df_clean2['cum_LOSSES_away_H2H'] = cum_losses_away_H2H
df_clean2['cum_5_WINS_away_H2H'] = cum_5_wins_away_H2H
df_clean2['cum_10_WINS_away_H2H'] = cum_10_wins_away_H2H
df_clean2['cum_5_LOSSES_away_H2H'] = cum_5_losses_away_H2H
df_clean2['cum_10_LOSSES_away_H2H'] = cum_10_losses_away_H2H


### Vegas Betting Odds

We then load in our ODDS datasets for the required seasons. This dataset required cleaning for the names and dates to make it usable to join with the rest of the cumulative statistics. We then merge DF_CLEAN2 with ODDS using the TIMESTAMP and HOME_TEAM_NAME as keys, losing a few rows along the way due to missing data. 

In [44]:
# Cleaning the Odds DataFrame 

cols = [col for col in df_clean2.columns if np.any(['5' in col, '10' in col])]
for col in cols: 
    new_col = re.sub(r'_\d{1,2}_', '_', col)
    df_clean2[col] = df_clean2[col].fillna(df_clean2[new_col])


odds = pd.DataFrame()
for season in season_list: 
    df = pd.read_excel('ODDS_DATA/odds_' + str(season) + '.xlsx', usecols = ['Date', 'Team', 'ML'])
    df['Date'] = df['Date'].apply(lambda x: '{0:0>4}'.format(x))

    new_year_index = df[df['Date'].str.contains(r'01\d{2}')].index[0]
    year1 = np.repeat(str(season), new_year_index-1)
    year2 = np.repeat(str(season+1), len(df) - len(year1))
    df['Year'] = np.append(year1, year2)
    df['TIMESTAMP'] = df['Year'] + df['Date']
    df = df.drop('Date', axis = 1)
    odds = odds.append(df)
home_teams, home_odds = odds.iloc[1::2]['Team'], odds.iloc[1::2]['ML']
away_teams, away_odds = odds.iloc[::2]['Team'], odds.iloc[::2]['ML']
dates = odds.iloc[1::2]['TIMESTAMP']
odds = pd.DataFrame({'TIMESTAMP' : list(dates), 
                   'HOME_TEAM_NAME' : list(home_teams),
                   'AWAY_TEAM_NAME' : list(away_teams), 
                   'HOME_TEAM_ODDS' : list(home_odds), 
                   'AWAY_TEAM_ODDS' : list(away_odds)})
odds['HOME_TEAM_NAME'] = odds['HOME_TEAM_NAME'].str.replace(r'([a-z])([A-Z])', r'\1 \2')
odds['AWAY_TEAM_NAME'] = odds['AWAY_TEAM_NAME'].str.replace(r'([a-z])([A-Z])', r'\1 \2')

odds['TIMESTAMP'] = pd.to_datetime(odds['TIMESTAMP'], format = '%Y%m%d', errors = 'coerce')
teams = pd.read_csv('FP1_DATA/teams.csv', usecols = ['CITY', 'NICKNAME'])
teams['CITY'][7] = 'LAClippers'
teams['CITY'][8] = 'LALakers'

odds['HOME_TEAM_NAME'] = odds['HOME_TEAM_NAME'].str.replace('New Jersey', 'Brooklyn')
odds['AWAY_TEAM_NAME'] = odds['AWAY_TEAM_NAME'].str.replace('New Jersey', 'Brooklyn')

team_city_dict = teams.set_index('CITY')['NICKNAME'].T.to_dict()
odds['HOME_TEAM_NAME'] = odds['HOME_TEAM_NAME'].map(team_city_dict)
odds['AWAY_TEAM_NAME'] = odds['AWAY_TEAM_NAME'].map(team_city_dict)

new_df = pd.merge(df_clean2, odds, on = ['TIMESTAMP', 'HOME_TEAM_NAME'])

cols = [col for col in new_df.columns if np.any(['5' in col, '10' in col])]
for col in cols: 
    new_col = re.sub(r'_\d{1,2}_', '_', col)
    new_df[col] = new_df[col].fillna(new_df[new_col])

new_df = new_df.fillna(0)
new_df['SEASON'] = new_df['GAME_ID'].map(season_dict)

cum_col_list = np.append([col for col in list(new_df.columns) if 'cum' in col], ['HOME_TEAM_WINS', 'SEASON', 'GAME_ID'])
new_df = new_df[cum_col_list]


Lastly, we convert this final dataframe to a CSV to load into our FP3_MODELS notebook.

In [45]:
new_df.to_csv('FP1.csv', index = False)

In [47]:
new_df.shape

(15465, 819)